In [ ]:
import numpy as np
# from scipy.optimize import curve_fit
import sys
import matplotlib.pyplot as plt

import iminuit
print("iminuit version:", iminuit.__version__)
from iminuit import Minuit
from iminuit.cost import LeastSquares
import inspect

In [ ]:
## PARAMETERS
# Computing B1 LS
# B1 is stationary
# B2 is scanning

# definition of LS: BPM_length * LS = true_length

params = {
    # "Nominal_LS":1.05,
    "Nominal_LS_B1":1.005,
    # "Nominal_LS_B1":0.995,
    # "Doros_LS_B1":1.05,
    # "Doros_LS_B1":1.0,
    "Doros_LS_B1":0.995,
    # "Arc_LS_B1":1.05,
    "Arc_LS_B1":1.0,
    # "Arc_LS_B1":0.995,
    "Nominal_LS_B2":1.0,
    # "Nominal_LS_B2":0.995,
    # "Doros_LS_B2":1.05,
    "Doros_LS_B2":0.99,
    # "Doros_LS_B2":0.995,
    # "Arc_LS_B2":1.05,
    "Arc_LS_B2":1.0,
    # "Arc_LS_B2":0.995,
    "Sigma_B1":90, #micron
    "Sigma_B2":90, #micron
    "nominal_step_size_B1": 130, #micron
    "nominal_step_size_B2": 130, #micron
    "cumulative_OD":False,
    "OD_B1":[
        10,0,0,
        0,0,0,
        0,0,0,
        0,0,0,
        0,0,0,
    ],
    "OD_B2":[
        0,0,0,
        0,0,0,
        0,0,0,
        0,0,0,
        0,0,0,
    ],
    "Doros_noise_sigma":0, # micron
    "Arc_noise_sigma":0, # micron
    "rate_runcertainty_at_head_on":0.005, 
    "beamspot_position_uncertainty":1, #micron
    # "pos_strategy":"doros, middle point",
    "pos_strategy":"doros, avg point",
    # "pos_strategy":"doros, b1 stepsize outlier1, sep stepsize 10 outlier4, avg point",
}

np.random.seed(662)

sigma_OD = 10
od_b1 = np.random.normal(size=15)*sigma_OD
od_b2 = np.random.normal(size=15)*sigma_OD
# idx = 10#int(np.random.random()*15)
# print(idx)
# od_b1[idx] += 10

params["OD_B1"]= od_b1
params["OD_B2"]= od_b2
params["cumulative_OD"]=True

In [ ]:
# utility fitting function using minuit for better error propagation
def curve_fit(f, x, y, sigma=None, p0=None, p0_lim=None):

    if sigma is None:
        sigma = np.ones_like(y)

    init_params = {}
    if not p0 is None:
        for i, n in enumerate(inspect.getfullargspec(f)[0][1:]):
            init_params[n]=p0[i]
    if not p0_lim is None:
        for i, n in enumerate(inspect.getfullargspec(f)[0][1:]):
            init_params["limit_"+n]=p0_lim[i]

    least_squares = LeastSquares(x, y, sigma, f)

    m = Minuit(least_squares, **init_params)

    m.migrad()    
    try:
        cov = np.array(m.matrix())
    except:
        try:
            m.hesse()
            cov = np.array(m.matrix())
        except:
            m.hesse()
            cov = np.array(m.matrix())
        

    return m.np_values(), cov



# test
def lin(x,a,b):
        return a*x+b
popt_lin, pcov_lin = curve_fit(lin, range(4), [0,1.1,1.9,3], sigma=[1,1,1,1], p0=[1, 0])
print(popt_lin)
print(pcov_lin)

In [ ]:
## nominal beampositions

nominal_B1_positions = np.repeat((np.arange(5)*params["nominal_step_size_B1"]),3)
scan_pattern = np.tile([-params["nominal_step_size_B2"], 0, params["nominal_step_size_B2"]], 5)
nominal_B2_positions = nominal_B1_positions + scan_pattern

plt.plot(range(15), nominal_B1_positions, "or")
plt.plot(range(15), nominal_B2_positions, "xb")
plt.xlabel('Tick')
plt.ylabel('Beam position')

In [ ]:
## OD and BPM
if params["cumulative_OD"]:
    true_B1_positions = nominal_B1_positions * params["Nominal_LS_B1"] + np.cumsum(params["OD_B1"])
    true_B2_positions = nominal_B2_positions * params["Nominal_LS_B2"] + np.cumsum(params["OD_B2"])
else:
    true_B1_positions = nominal_B1_positions * params["Nominal_LS_B1"] + params["OD_B1"]
    true_B2_positions = nominal_B2_positions * params["Nominal_LS_B2"] + params["OD_B2"]

doros_B1_positions = true_B1_positions / params["Doros_LS_B1"] + np.random.normal(size=true_B1_positions.shape) * params["Doros_noise_sigma"] 
doros_B2_positions = true_B2_positions / params["Doros_LS_B2"] + np.random.normal(size=true_B2_positions.shape) * params["Doros_noise_sigma"]

arc_B1_positions = true_B1_positions / params["Arc_LS_B1"] + np.random.normal(size=true_B1_positions.shape) * params["Arc_noise_sigma"]
arc_B2_positions = true_B2_positions / params["Arc_LS_B2"] + np.random.normal(size=true_B2_positions.shape) * params["Arc_noise_sigma"]

avg_B1_positions = (doros_B1_positions + arc_B1_positions)/2
avg_B2_positions = (doros_B2_positions + arc_B2_positions)/2


plt.plot(range(15), doros_B1_positions-nominal_B1_positions+1, "o-r")
plt.plot(range(15), arc_B1_positions-nominal_B1_positions-1, "o-b")
plt.plot(range(15), avg_B1_positions-nominal_B1_positions, "o-y")
plt.xlabel('Tick')
plt.ylabel('BPM - nominal')
plt.show()

plt.plot(range(15), doros_B2_positions-nominal_B2_positions+1, "o-r")
plt.plot(range(15), arc_B2_positions-nominal_B2_positions-1, "o-b")
plt.plot(range(15), avg_B2_positions-nominal_B2_positions, "o-y")
plt.xlabel('Tick')
plt.ylabel('BPM - nominal')
plt.show()


In [ ]:
def avg_stepsize_corr_b1(positions, nominal_positions, b1_strategy = "m", outliers=0, plot=False):

    nominal_step_size = nominal_positions[3] - nominal_positions[0]

    if b1_strategy == "c":
        pos_c   = positions[1::3]        
    elif b1_strategy == "m":
        pos_c = np.array( [ np.mean(positions[3*i:3*i+3]) for i in range(5)] ) 
    diffs =  np.diff( pos_c )

    if plot:
        plt.plot(range(len(diffs)), diffs, "o-b")

    if outliers == 0:
        avg_step_size = np.mean( diffs )
    else:
        excluded_indices = []

        diffs1  = diffs[:]
        for i in range(outliers):
            avg     = np.mean( diffs1 )
            i_extremum = np.argmax(np.abs(diffs1-avg))
            if np.abs(diffs1[i_extremum] - avg)<0.01:
                break
            excluded_indices.append( np.where(np.isclose(diffs, diffs1[i_extremum]))[0][0] )
            diffs1 = diffs1[ np.arange(len(diffs1)) != i_extremum ]
        avg = np.mean(diffs1)
        std = np.std(diffs1)
        lim = 3*std
        diffs2 = diffs[ np.abs( diffs-avg )<=lim ]
        avg_step_size = np.mean(diffs2)

        if plot:
            if len(excluded_indices)>0:
                plt.plot(excluded_indices, diffs[np.array(excluded_indices)], "xr", markersize=12)
            plt.plot([-1,5], [avg, avg], "-", color="red")
            plt.fill_between([-1,5], [avg-lim, avg-lim], [avg+lim, avg+lim], color="red", alpha=0.2)
            plt.plot(np.arange(len(diffs))[ np.abs( diffs-avg )>lim ]+0.3, diffs[ np.abs( diffs-avg )>lim ], "*m", markersize=12)
    
    if plot:
        print("avg_step_size measured:", avg_step_size)
        plt.plot([-1,5], [avg_step_size, avg_step_size], "--g")
        plt.show()

            
    od_estimates = diffs - avg_step_size
    new_stepsizes = od_estimates + nominal_step_size
    new_positions = np.concatenate(([0], np.cumsum(new_stepsizes)))
    # take the middle point as reference
    new_positions = new_positions - new_positions[2] + nominal_positions[7]

    return new_positions

# testing
# test_positions = np.repeat(np.arange(5),3)*130.0
# test_positions[0:2]+=10
# print(avg_stepsize_corr_b1(test_positions, nominal_B1_positions, b1_strategy = "c", outliers=1, plot=True))

# test_positions = np.repeat(np.arange(5),3)*130.0
# test_positions+= np.random.normal(size=15)*10
# print(avg_stepsize_corr_b1(test_positions, nominal_B1_positions, b1_strategy = "m", outliers=1, plot=True))


print(avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "m", outliers=2, plot=True))
print("avg_step_size doros, theor.:", params["nominal_step_size_B1"]*params["Nominal_LS_B1"] / params["Doros_LS_B1"])

In [ ]:
def avg_stepsize_corr_sep(positions, nominal_positions, steps_to_average=10, outliers=0, plot=False, reference=1):

    nominal_step_size = nominal_positions[1] - nominal_positions[0]

    diffs = np.diff(positions)
    diffs = diffs[ (np.arange(len(diffs)) % 3) != 2]      

    if plot:
        print("diffs", diffs)
        plt.plot(range(len(diffs)), diffs, "o-b")

    if steps_to_average == 10:
        if outliers == 0:
            avg_step_size = np.mean( diffs )
        else:
            # remove 4 outliers
            excluded_indices = []
            diffs1  =diffs[:]
            for i in range(outliers):
                avg  = np.mean( diffs1 )
                i_extremum = np.argmax(np.abs(diffs1-avg))
                if np.abs(diffs1[i_extremum] - avg)<0.01:
                    break
                excluded_indices.append( np.where(np.isclose(diffs, diffs1[i_extremum]))[0][0] )
                diffs1 = diffs1[ np.arange(len(diffs1)) != i_extremum ]
            avg = np.mean(diffs1)
            std = np.std(diffs1)
            lim = 3*std
            diffs2 = diffs[ np.abs( diffs-avg )<=lim ]
            avg_step_size = np.mean(diffs2)


            if plot:
                if len(excluded_indices)>0:
                    plt.plot(excluded_indices, diffs[np.array(excluded_indices)], "xr", markersize=12)
                plt.plot([-1,11], [avg, avg], "-", color="red")
                plt.fill_between([-1,11], [avg-lim, avg-lim], [avg+lim, avg+lim], color="red", alpha=0.2)
                plt.plot(np.arange(len(diffs))[ np.abs( diffs-avg )>lim ]+0.3, diffs[ np.abs( diffs-avg )>lim ], "*m", markersize=12)
                
        if plot:
            print("avg_step_size measured:", avg_step_size)
            plt.plot([-1,11], [avg_step_size, avg_step_size], "--g")
            plt.show()

        od_estimates = diffs - avg_step_size
        # od_estimates *= nominal_step_size/avg_step_size
        new_stepsizes = od_estimates + nominal_step_size

    elif steps_to_average == 2:

        avg_step_sizes = np.repeat([ np.mean(diffs[2*i:2*i+2]) for i in range(5) ], 2)
        od_estimates = diffs - avg_step_sizes
        new_stepsizes = od_estimates + nominal_step_size

        if plot:
            print("avg_step_sizes measured:", avg_step_sizes)
            for i in range(5):
                plt.plot(np.arange(len(avg_step_sizes))[2*i:2*i+2], avg_step_sizes[2*i:2*i+2], "--g")
            plt.show()



    if reference == 1:
        # reference is always the middle point
        if plot:
            print("od_estimates", od_estimates)
            print("new stepsizes", new_stepsizes)
        result = np.repeat(nominal_positions[1::3], 3)*1.0
        for i in range(len(result)):
            if i%3==0:
                result[i] -= new_stepsizes[i//3*2]
            elif i%3==2:
                result[i] += new_stepsizes[(i-2)//3*2+1]
        return result

    elif reference == 2:
        return positions - avg_step_size * nominal_positions / nominal_step_size + nominal_positions
    

# test_positions = np.array(nominal_B2_positions)*1.0
# test_positions[0]+=10
# print(avg_stepsize_corr_sep(test_positions, nominal_B2_positions, steps_to_average=10, outliers=4, plot=True))

# test_positions = np.array(nominal_B2_positions)*1.0
# test_positions+= np.cumsum(np.random.normal(size=15)*10)
# print(avg_stepsize_corr_sep(test_positions, nominal_B2_positions, steps_to_average=10, outliers=4, plot=True))

# test_positions = np.array(nominal_B2_positions)*1.0
# test_positions+= np.cumsum(np.random.normal(size=15)*10)
# print(avg_stepsize_corr_sep(test_positions, nominal_B2_positions, steps_to_average=2, outliers=0, plot=True))

doros_sep   = doros_B1_positions - doros_B2_positions
nominal_sep = nominal_B1_positions - nominal_B2_positions
print("sep doros",doros_sep)
corr_sep = avg_stepsize_corr_sep(doros_sep, nominal_sep, steps_to_average=10, outliers=4, plot=True)
print( "corr_sep", corr_sep )
doros_sep_in_nominal_dims = doros_sep/params["Nominal_LS_B1"] * params["Doros_LS_B1"]
print("doros_sep_in_nominal_dims", doros_sep_in_nominal_dims)

print("first half differences", np.diff(corr_sep)[0::3] - np.diff(doros_sep_in_nominal_dims)[0::3])
print("second half differences",np.diff(corr_sep)[1::3] - np.diff(doros_sep_in_nominal_dims)[1::3])

print("avg_step_size doros, theor.:", params["nominal_step_size_B2"]*params["Nominal_LS_B2"] / params["Doros_LS_B2"])


In [ ]:
## OD correction of nominal

# this part needs to compute the followings:
# analysis_B1_positions or analysis_separations
# analysis_B2_positions or analysis_separations
# analysis_B1_5positions

# maybe we don't need to do step size correction for the scanning beam at all! 
# after all this method should be only sensitive to the LS of B1!
# in fact, probably we should always just use avg for the scanning beam, as that is the safest and most accrate probably

# the miniscan deltas are from the bpm-s directly 

if params["pos_strategy"]=="nominal, middle point":
    analysis_separations   = nominal_B1_positions - nominal_B2_positions
    analysis_B1_5positions = nominal_B1_positions[1::3]
elif params["pos_strategy"]=="nominal, avg point":
    analysis_separations   = nominal_B1_positions - nominal_B2_positions
    analysis_B1_5positions = np.array([ np.mean(nominal_B1_positions[3*i:3*i+3]) for i in range(5)])
elif params["pos_strategy"]=="doros, middle point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_B1_5positions = doros_B1_positions[1::3]
elif params["pos_strategy"]=="doros, avg point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_B1_5positions = np.array([ np.mean(doros_B1_positions[3*i:3*i+3]) for i in range(5)])
#######################
elif params["pos_strategy"]=="doros, b1 stepsize, sep stepsize 10, avg point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=0)
    analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "m", outliers=0)
elif params["pos_strategy"]=="doros, b1 stepsize, sep stepsize 2, avg point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=2, outliers=0)
    analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "m", outliers=0)
elif params["pos_strategy"]=="doros, b1 stepsize, sep stepsize 10 outlier4, avg point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=4)
    analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "m", outliers=0)
elif params["pos_strategy"]=="doros, b1 stepsize outlier 1, sep stepsize 10, avg point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=0)
    analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "m", outliers=1)
elif params["pos_strategy"]=="doros, b1 stepsize outlier1, sep stepsize 10 outlier4, avg point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=4)
    analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "m", outliers=2)
elif params["pos_strategy"]=="doros, b1 stepsize outlier1, sep stepsize 10 outlier2, avg point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=2)
    analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "m", outliers=1)
#######################
elif params["pos_strategy"]=="doros, b1 stepsize, sep stepsize 10, middle point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=0)
    analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=0)
elif params["pos_strategy"]=="doros, b1 stepsize, sep stepsize 2, middle point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=2, outliers=0)
    analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=0)
elif params["pos_strategy"]=="doros, b1 stepsize, sep stepsize 10 outlier4, middle point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=4)
    analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=0)
elif params["pos_strategy"]=="doros, b1 stepsize outlier 1, sep stepsize 10, middle point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=0)
    analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=1)
elif params["pos_strategy"]=="doros, b1 stepsize outlier1, sep stepsize 10 outlier4, middle point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=4)
    analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=1)
elif params["pos_strategy"]=="doros, b1 stepsize outlier1, sep stepsize 10 outlier2, middle point":
    analysis_separations   = doros_B1_positions - doros_B2_positions
    analysis_separations   = avg_stepsize_corr_sep(analysis_separations, nominal_B2_positions, steps_to_average=10, outliers=2)
    analysis_B1_5positions = avg_stepsize_corr_b1(doros_B1_positions, nominal_B1_positions, b1_strategy = "c", outliers=1)

In [ ]:
## beamspot location 
true_beamspot_positions = ( 
        true_B1_positions/params["Sigma_B1"]**2 + 
        true_B2_positions/params["Sigma_B2"]**2
    )/(params["Sigma_B1"]**-2+params["Sigma_B2"]**-2)


plt.plot(range(15), true_B1_positions, "or")
plt.plot(range(15), true_B2_positions, "xb")
plt.plot(range(15), true_beamspot_positions, "xg")
plt.xlabel('Tick')
plt.ylabel('Beam position / bs position')

In [ ]:
## rates
true_rates     = np.exp(-0.5 * (true_B1_positions-true_B2_positions)**2 / (params["Sigma_B1"]**2+params["Sigma_B2"]**2) )
true_rates_unc = true_rates**0.5 * np.max(true_rates)**0.5 * params["rate_runcertainty_at_head_on"]

In [ ]:
## calculate head-on beamspot position

def get_headon_position(rates, rate_unc, separations, beamspot_positions ):

    def sg(x,a,m,s):
        return a*np.exp(-0.5*((x-m)/s)**2)
    
    a0 = np.max(rates)
    m0 = separations[1]
    s0 = np.abs(separations[1] - separations[0])
    popt_sg, pcov_sg = curve_fit(sg, separations, rates, sigma=rate_unc, 
                                    p0=[a0, m0, s0 ],
                                    p0_lim = [
                                        [a0/2, a0*2],
                                        [m0-s0*2, m0+s0*2],
                                        [s0/2, s0*2],
                                    ] )

    mu               = popt_sg[1]

    def lin(x,a,b):
        return a*x+b

    popt_lin, pcov_lin = curve_fit(lin, separations, beamspot_positions, sigma=[params["beamspot_position_uncertainty"]]*3, p0=[1, 0])
    a = popt_lin[0]
    b = popt_lin[1]

    # compute result uncertainty
    if np.allclose(pcov_lin, 0):
        std = a*pcov_sg[1,1]**0.5
    else:
        mu_randomized = mu + pcov_sg[1,1]**0.5*np.random.normal(size=1000)
        try:
            pcov_lin_sqrt = np.linalg.cholesky(pcov_lin)
        except Exception as e:
            print(e)
            print(pcov_lin)
            sys.exit()

        ab_randomized = popt_lin[:,np.newaxis] + np.einsum("ij,jk", pcov_lin_sqrt, np.random.normal(size=(2,1000)))
        std = np.std( ab_randomized[0,:]*mu_randomized + ab_randomized[1,:] )

    if std == 0:
        std = 1e-6


    fig, ax1 = plt.subplots()
 
    color = 'tab:red'
    ax1.set_xlabel('sep')
    ax1.set_ylabel('Rate', color = color)
    ax1.errorbar(separations, rates, yerr=rate_unc, fmt="o", color = color)
    x = np.linspace(separations[0] - (separations[-1]-separations[0])*0.2, separations[-1] + (separations[-1]-separations[0])*0.2, 100)
    ax1.plot(x, sg(x, *popt_sg ), color = color)
    ax1.tick_params(axis ='y', labelcolor = color)

    ax1.plot([mu, mu], [min(rates)/1.1, max(rates)*1.1], "k--")
    
    # Adding Twin Axes to plot using dataset_2
    ax2 = ax1.twinx()
    
    color = 'tab:green'
    ax2.set_ylabel('Beamspot positions', color = color)
    ax2.errorbar(separations, beamspot_positions, yerr=[params["beamspot_position_uncertainty"]]*3, fmt="o", color = color)
    ax2.plot(x, lin(x, *popt_lin ), color = color)
    ax2.tick_params(axis ='y', labelcolor = color)

    ax2.plot([min(separations), max(separations)], [a*mu+b]*2, "k--")

    return a*mu+b, std


computed_head_on_bs_position = []
computed_head_on_bs_position_unc = []
for i in range(5):

    v, u = get_headon_position(
        true_rates[3*i:3*i+3],
        true_rates_unc[3*i:3*i+3],
        # analysis_B1_positions[3*i:3*i+3] - analysis_B2_positions[3*i:3*i+3],
        analysis_separations[3*i:3*i+3],
        true_beamspot_positions[3*i:3*i+3])
    computed_head_on_bs_position.append(v) 
    computed_head_on_bs_position_unc.append(u)

print(computed_head_on_bs_position)
print(computed_head_on_bs_position_unc)

In [ ]:
# analysis_B1_5positions = analysis_B1_5positions[:-1]
# computed_head_on_bs_position = computed_head_on_bs_position[:-1]
# computed_head_on_bs_position_unc = computed_head_on_bs_position_unc[:-1]

In [ ]:

def lin(x,a,b):
        return a*x+b


a0 = (computed_head_on_bs_position[-1]-computed_head_on_bs_position[0])/(analysis_B1_5positions[-1]-analysis_B1_5positions[0])
b0 = 0
popt_lin, pcov_lin = curve_fit(lin, analysis_B1_5positions, computed_head_on_bs_position, sigma=computed_head_on_bs_position_unc, p0=[a0, b0])
a = popt_lin[0]
b = popt_lin[1]

print("computed LSC", a, "+/-", pcov_lin[0,0]**0.5)

liml = np.min(analysis_B1_5positions) - 0.2 * (np.max(analysis_B1_5positions)-np.min(analysis_B1_5positions))
limu = np.max(analysis_B1_5positions) + 0.2 * (np.max(analysis_B1_5positions)-np.min(analysis_B1_5positions))
x = np.linspace(liml, limu, 2)
plt.plot(x, lin(x,a-1,b), "b-")
plt.errorbar(analysis_B1_5positions, computed_head_on_bs_position - analysis_B1_5positions, yerr=computed_head_on_bs_position_unc, fmt="or")


plt.xlabel('"'+params["pos_strategy"] + '" position')
plt.ylabel('Beamspot position - X values')